# π Estimation with Monte Carlo methods
We demonstrate how to run Monte Carlo simulations with lithops over IBM Cloud Functions. This notebook contains an example of estimation the number π with Monte Carlo. The goal of this notebook is to demonstrate how IBM Cloud Functions can benefit Monte Carlo simulations and not how it can be done using lithops.<br>
A Monte Carlo algorithm would randomly place points in the square and use the percentage of randomized points inside of the circle to estimate the value of π
![pi](https://upload.wikimedia.org/wikipedia/commons/8/84/Pi_30K.gif)
Requirements to run this notebook:

* AWS Cloud or GCP account. 
* You will need to have at least one existing object storage bucket. 

# Step 1 - Install dependencies
Install dependencies

In [35]:
from time import time
from random import random
import logging
import sys

try:
    import lithops
except:
    %pip install -r requirements.txt
    import lithops

# you can modify logging level if needed
#logging.basicConfig(level=logging.INFO)

# Step 2 - Write Python code that implements Monte Carlo simulation 
Below is an example of Python code to demonstrate Monte Carlo model for estimate PI

'EstimatePI' is a Python class that we use to represent a single PI estimation. You may configure the following parameters:

MAP_INSTANCES - number of cloud functions invocations. Default is 100<br>
randomize_per_map - number of points to random in a single invocation. Default is 10,000,000

Our code contains two major Python methods:

def randomize_points(self,data=None) - a function to random number of points and return the percentage of points
    that inside the circle<br>
def process_in_circle_points(self, results, futures): - summarize results of all randomize_points
  executions (aka "reduce" in map-reduce paradigm)

In [36]:
MAP_INSTANCES = 50


class EstimatePI:
    randomize_per_map = 10000000

    def __init__(self):
        self.total_randomize_points = MAP_INSTANCES * self.randomize_per_map

    def __str__(self):
        return "Total Randomize Points: {:,}".format(self.randomize_per_map * MAP_INSTANCES)

    @staticmethod
    def predicate():
        x = random()
        y = random()
        return (x ** 2) + (y ** 2) <= 1

    def randomize_points(self, data):
        in_circle = 0
        for _ in range(self.randomize_per_map):
            in_circle += self.predicate()
        return float(in_circle / self.randomize_per_map)

    def process_in_circle_points(self, results):
        in_circle_percent = 0
        for map_result in results:
            in_circle_percent += map_result
        estimate_PI = float(4 * (in_circle_percent / MAP_INSTANCES))
        return estimate_PI

# Step 3 - Configure access to your Cloud Storage and Cloud Functions

Configure access details to your AWS or other cloud provider.  'storage_bucket'  should point to some pre-existing bucket. This bucket will be used by Lithops to store intermediate results. All results will be stored in the folder `lithops.jobs`.

e.g. for GCP your `.lithops_config` should be similar to: 
    
    lithops:
        storage: gcp_storage
        backend: gcp_functions
        bucket: lithops-pipelines
    
    gcp:
        credentials_path : <PATH_TO_JSON_KEYS>
        region : <GCP_REGION>
    
    gcp_functions:
        region : <GCP_REGION>
    
    gcp_storage:
        region: <GCP_REGION>
        storage_bucket: <GCP_STORAGE_BUCKET>

For AWS your `.lithops_config` should be similar to: 
    
    lithops:
        storage: aws_s3
        backend: aws_lambda
    
    aws:
        access_key_id : <AWS_ACCESS_KEY_ID>
        secret_access_key : <AWS_SECRET_ACCESS_KEY> 
        
    aws_s3:
        storage_bucket: <S3_BUCKET>
        region_name : <REGION>
    
    aws_lambda:
        execution_role: <AWS_ROLE_ARN>
        region_name: <REGION>

# Step 4 - Execute simulation with Lithops over IBM Cloud Functions 

In [37]:
iterdata = [0] * MAP_INSTANCES # funcion + iterable --> length ( numero de elementos )
est_pi = EstimatePI()

start_time = time()
print("Monte Carlo simulation for estimating PI spawing over {} Cloud Function invocations".format(MAP_INSTANCES))
# obtain lithops executor
pw = lithops.FunctionExecutor(runtime_memory=2048)

# execute the code
pw.map_reduce(est_pi.randomize_points, iterdata, est_pi.process_in_circle_points) # iterdata : 
#get results
result = pw.get_result()
elapsed = time()
print(str(est_pi))
print("Estimation of Pi: ", result)
print("\nCompleted in: " + str(elapsed - start_time) + " seconds")

2025-03-21 10:34:41,216 [INFO] config.py:139 -- Lithops v3.6.1.dev0 - Python3.12
2025-03-21 10:34:41,216 [INFO] localhost.py:39 -- Localhost storage client created
2025-03-21 10:34:41,217 [INFO] localhost.py:78 -- Localhost compute v2 client created
2025-03-21 10:34:41,227 [INFO] invokers.py:119 -- ExecutorID 380724-8 | JobID M000 - Selected Runtime: python - 2048MB
2025-03-21 10:34:41,232 [INFO] invokers.py:186 -- ExecutorID 380724-8 | JobID M000 - Starting function invocation: randomize_points() - Total: 50 activations
2025-03-21 10:34:41,253 [INFO] invokers.py:225 -- ExecutorID 380724-8 | JobID M000 - View execution logs at /tmp/lithops-bigrobbin/logs/380724-8-M000.log
2025-03-21 10:34:41,258 [INFO] wait.py:105 -- ExecutorID 380724-8 - Waiting for 20% of 50 function activations to complete


Monte Carlo simulation for estimating PI spawing over 50 Cloud Function invocations


    0%|          | 0/10  

2025-03-21 10:34:44,517 [INFO] invokers.py:119 -- ExecutorID 380724-8 | JobID R000 - Selected Runtime: python - 2048MB
2025-03-21 10:34:44,522 [INFO] invokers.py:186 -- ExecutorID 380724-8 | JobID R000 - Starting function invocation: process_in_circle_points() - Total: 1 activations
2025-03-21 10:34:44,523 [INFO] invokers.py:225 -- ExecutorID 380724-8 | JobID R000 - View execution logs at /tmp/lithops-bigrobbin/logs/380724-8-R000.log
2025-03-21 10:34:44,523 [INFO] executors.py:494 -- ExecutorID 380724-8 - Getting results from 1 function activations
2025-03-21 10:34:44,523 [INFO] wait.py:101 -- ExecutorID 380724-8 - Waiting for 41 function activations to complete


    0%|          | 0/41  

2025-03-21 10:34:48,233 [INFO] executors.py:618 -- ExecutorID 380724-8 - Cleaning temporary data


Total Randomize Points: 500,000,000
Estimation of Pi:  3.1424403999999986

Completed in: 7.019773960113525 seconds


In [38]:
# Price in AWS
import numpy as np

stats = [f.stats for f in pw.futures]
mean_exec_time = np.mean([stat['worker_func_exec_time'] for stat in stats])

# Debug: Print the structure of worker_func_perf_energy to understand what's in it
if stats and 'worker_func_perf_energy' in stats[0]:
    print(f"Structure of worker_func_perf_energy: {stats[0]['worker_func_perf_energy']}")

# Handle worker_func_perf_energy as a dictionary
# If it's a dictionary, we need to extract a specific value or calculate an aggregate
try:
    # Option 1: If there's a specific key we want to extract from each dictionary
    # For example, if each dictionary has a 'total' or 'value' key
    if stats and 'worker_func_perf_energy' in stats[0] and isinstance(stats[0]['worker_func_perf_energy'], dict):
        # Try to find a numeric key in the dictionary
        sample_dict = stats[0]['worker_func_perf_energy']
        numeric_keys = [k for k, v in sample_dict.items() if isinstance(v, (int, float))]
        
        if numeric_keys:
            # Use the first numeric key found
            key_to_use = numeric_keys[0]
            worker_func_perf_energy = np.mean([stat['worker_func_perf_energy'][key_to_use] 
                                              for stat in stats 
                                              if 'worker_func_perf_energy' in stat 
                                              and key_to_use in stat['worker_func_perf_energy']])
            print(f"Using key '{key_to_use}' from worker_func_perf_energy dictionary")
        else:
            # If no numeric keys found, skip this calculation
            worker_func_perf_energy = "N/A (No numeric values found in dictionary)"
    else:
        # If it's not a dictionary or doesn't exist, skip this calculation
        worker_func_perf_energy = "N/A"
        
except Exception as e:
    print(f"Error processing worker_func_perf_energy: {e}")
    worker_func_perf_energy = "N/A (Error)"

print(f"Perf Energy: {worker_func_perf_energy}")
# print(f"Global Perf Energy: {pw.stats.get('worker_func_perf_energy', 'N/A')}")

gbxms_price = 0.0000000167
sum_total_time = sum([stat['worker_exec_time'] for stat in stats]) * 1000
price = gbxms_price * sum_total_time * 0.256  # Price GB/ms * sum of times in ms * 0.256 GB runtime

print(f'Experiment total price is {round(price, 5)} USD')

Structure of worker_func_perf_energy: {'pkg': 609.52, 'cores': 590.26, 'total': 1199.78}
Using key 'pkg' from worker_func_perf_energy dictionary
Perf Energy: 560.6576470588235
Experiment total price is 0.00061 USD
